In [20]:
import random

In [91]:
class Value():
    def __init__(self, value, _children=()):
        self.value = value
        self.gradient = 0

        self._backward = lambda: None
        self._previous = set(_children)
    
    def __add__(self, other):
        other = other if isinstance(other,Value) else Value(other)
        out = Value(self.value + other.value, (self,other))

        def _backward():
            self.gradient += out.gradient
            other.gradient += out.gradient
        
        out._backward = _backward
        return out
    
    def __radd__(self, other): 
        # use __add__, i.e. other + self for right addition
        return self + other
    
    def __mul__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out = Value(self.value * other.value, (self,other))

        def _backward():
            self.gradient = other.value * out.gradient
            other.gradient = self.value * out.gradient 

        out._backward = _backward
        return out
    
    def __pow__(self,other):
        out = Value(self.value**other , (self,))

        def _backward():
            self.gradient = (other *self.value**(other-1)) * self.gradient
        out._backward = _backward
        return out

    def relu(self):
        out = Value((self.value>0)* self.value, (self,) )

        def _backward():
            self.gradient = (self.value>0)*self.gradient
        out._backward = _backward
        return out

In [76]:
class Module:
    def zero_gradient(self):
        for param in self.parameters():
            param.gradient = 0

In [77]:
class Neuron(Module):
    def __init__(self, n_input, is_nonlinear=True):
        self.weigths = [Value(random.uniform(-1,1)) for _ in range(n_input)]
        self.bias = Value(0)
        self.is_nonlinear = is_nonlinear
    def __call__(self, x):
        activation = sum(weight_i*x_i for weight_i, x_i in zip(self.weigths,x))
        return activation.relu() if self.is_nonlinear else activation


In [78]:
class Layer(Module):
    def __init__(self, n_inputs, n_outputs, **kwargs):
        self.neurons = [Neuron(n_inputs, **kwargs) for _ in range(n_outputs)]

    def __call__(self, x):
        out = [neuron(x) for neuron in self.neurons]
        return out[0] if len(out) == 1 else out



In [79]:
def mse(target, value):
    return sum([target[i]-value[i]**2 for i in range(len(target))])

In [87]:
class Network(Module):
    def __init__(self, n_inputs, layer_sizes,loss_function=mse):
        size = [n_inputs] + layer_sizes
        self.layers = [Layer(size[i], size[i+1], is_nonlinear= i != len(layer_sizes) -1) for i in range(len(layer_sizes))]
        self.loss_function= loss_function
        

    def forward(self,x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def loss(self, x, target):
        y_hat = self.forward(x)
        loss = Value(self.loss_function(target, y_hat))
        return loss

In [92]:
net = Network(4, [6,6,2])

In [93]:
[x.gradient for x in net.forward([1,2,3,4])]

[0, 0]

In [94]:
net.loss([1,2], [1,1])

TypeError: 'Value' object is not iterable